In [19]:
import numpy as np
import BigFunctions as BF
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
import uproot as ur
%matplotlib tk

In [20]:
#Reading the variables

input_path = "../data/bigdata_CFD_20092024_0001.root"
with ur.open(input_path) as file:
    outTree = file["DataTree"]
    label = outTree["label"].array(library = "np")
    time = outTree["time"].array(library = "np")*10**-6   #[µs]
    nrj = outTree["nrj"].array(library = "np")
    pileup = outTree["pileup"].array(library = "np")

print("Duration of the data acquisition:", max(time)*10**(-6)/3600, "hours")
print("Duration of the data acquisition:", max(time), "hours")
print("Number of events acquired:", len(label)/10**6, "million")
print("Number of events acquired with PM2:", len(label[label == 2])/10**6, "million")

Duration of the data acquisition: 64.80661879629267 hours
Duration of the data acquisition: 233303827666.6536 hours
Number of events acquired: 48.889578 million
Number of events acquired with PM2: 16.993348 million


In [21]:
#To see if there was aproblem during the data acquisition
#print(time[label == 1][-50:-1]*10**(-6)/3600)
#print(time[label == 2][-100:-1]*10**(-6)/3600)

In [22]:
#Correlation

corr_t = 10*10**-3   #Correlation window
corr_mask = (time[1:]-time[:-1]<corr_t) & (label[1:] != label[:-1])

correl_both = (np.insert(corr_mask,0,False))|(np.append(corr_mask,False))
del(corr_mask)

t = time[correl_both]
l = label[correl_both]
p = pileup[correl_both]
e = nrj[correl_both]

t1 = time[correl_both & (label==1)]
t2 = time[correl_both & (label==2)]
nrj1 = nrj[correl_both & (label==1)]
nrj2 = nrj[correl_both & (label==2)]

dt2 = t2[1:]-t2[:-1]
dt1 = t1[1:]-t1[:-1]

print("Number of events after correlation:       ", len(t)/10**6, "million")
print("Dead time:                                ", min(dt2), "µs")
print("Total number of pileups:                  ", len(pileup[pileup == True]))
print("Total number of pileups after correlation:", len(p[p == True]))
print("Number of events acquired with PM2 after correlation:", len(l[l == 2])/10**6, "million")

Number of events after correlation:        22.2967 million
Dead time:                                 0.158599853515625 µs
Total number of pileups:                   0
Total number of pileups after correlation: 0
Number of events acquired with PM2 after correlation: 11.14835 million


In [24]:
#Energy Analysis

#After correlation only
print('... After correlation only ...')

nrj_max = 300000

fig, ax1 = plt.subplots(1, sharex=True)
arr_1,xe_1,ye_1 = np.histogram2d(nrj1, nrj2, bins =(500,100))
pcm_1 = ax1.pcolormesh(xe_1, ye_1,arr_1.T, cmap='rainbow', norm=LogNorm(vmin=arr_1.min()+1, vmax=arr_1.max()))
ax1.set_xlabel('Energy 1 [Arbitrary Unit]')
ax1.set_ylabel('Energy 2 [Arbitrary Unit]')
ax1.set_title('2D Histogram\nEnergy 2 as Function of Energy 1')
ax1.grid()
fig.colorbar(pcm_1, ax=ax1, orientation='vertical', fraction=.1)
plt.show()

plt.figure()
plt.hist(nrj1[nrj1 < nrj_max], bins = 150, alpha = 0.5, label = "PM1")
plt.hist(nrj2[nrj2 < nrj_max], bins = 150, alpha = 0.5, label = "PM2")
plt.xlabel("Energy [Arbitrary Unit]")
plt.ylabel("Occurrences")
plt.grid()
plt.legend()
plt.show()

... After correlation only ...


In [25]:
#After correlation and cut on dt
print('... After correlation and cut on dt ...')

dt_max = 20
print("Number of events acquired with PM2 after correlation and cut on dt:", len(dt2[dt2 < 20])/10**6, "million")

#Muon and electron energy
plt.figure()
#plt.hist(nrj2[1:][(nrj2[1:]<nrj_max) & (dt2<dt_max)], bins=150, label="Second Event", alpha = 0.5)
#plt.hist(nrj2[:-1][(nrj2[:-1]<nrj_max)  & (dt2<dt_max)], bins=150, label="First Event", alpha = 0.5)
plt.hist(nrj2[(nrj2<nrj_max) & (np.append(dt2, np.nan)<dt_max)], bins=150, label="First Event", alpha = 0.5)
plt.hist(nrj2[(nrj2<nrj_max) & (np.insert(dt2, 0, np.nan)<dt_max)], bins=150, label="Second Event", alpha = 0.5)
plt.xlabel('Energy [Arbitraty Unit]')
plt.ylabel('Occurrences')
plt.legend()
plt.grid()
plt.show()

#Electron as function of dt
fig, ax1 = plt.subplots(1, sharex=True)
arr_1,xe_1,ye_1 = np.histogram2d(dt2[(nrj2[1:]<nrj_max) & (dt2<dt_max)], 
                                 nrj2[1:][(nrj2[1:]<nrj_max) & (dt2<dt_max)],
                                 bins =(500,100))
pcm_1 = ax1.pcolormesh(xe_1, ye_1,arr_1.T, cmap='rainbow', norm=LogNorm(vmin=arr_1.min()+1, vmax=arr_1.max()))
ax1.set_xlabel('$\Delta t$ [µs]')
ax1.set_ylabel('Second Event Energy\n[Arbitrary Unit]')
ax1.set_title('2D Histogram\nSecond Event Energy as function of $\Delta t$')
ax1.grid()
fig.colorbar(pcm_1, ax=ax1, orientation='vertical', fraction=.1)
plt.show()

#Muon as function of dt
fig, ax1 = plt.subplots(1, sharex=True)
arr_1,xe_1,ye_1 = np.histogram2d(dt2[(nrj2[:-1]<nrj_max) & (dt2<dt_max)], 
                                 nrj2[:-1][(nrj2[:-1]<nrj_max) & (dt2<dt_max)],
                                 bins =(500,100))
pcm_1 = ax1.pcolormesh(xe_1, ye_1,arr_1.T, cmap='rainbow', norm=LogNorm(vmin=arr_1.min()+1, vmax=arr_1.max()))
ax1.set_xlabel('$\Delta t$ [µs]')
ax1.set_ylabel('First Event Energy\n[Arbitrary Unit]')
ax1.set_title('2D Histogram\nFirst Event Energy as function of $\Delta t$')
ax1.grid()
fig.colorbar(pcm_1, ax=ax1, orientation='vertical', fraction=.1)
plt.show()

#Muon as function of electron

fig, ax1 = plt.subplots(1, sharex=True)
arr_1,xe_1,ye_1 = np.histogram2d(nrj2[1:][(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)],
                                 nrj2[:-1][(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)],  
                                 bins =(500,100))
pcm_1 = ax1.pcolormesh(xe_1, ye_1,arr_1.T, cmap='rainbow', norm=LogNorm(vmin=arr_1.min()+1, vmax=arr_1.max()))
ax1.set_xlabel('Second Event Energy\n[Arbitrary Unit]')
ax1.set_ylabel('First Event Energy\n[Arbitrary Unit]')
ax1.set_title('2D Histogram\nFirst Event Energy as Function of Second Event Energy')
ax1.grid()
ax1.plot(np.linspace(1000, 250000, 10000),np.linspace(1000,250000, 10000),c='r')
ax1.set_xscale("log")
fig.colorbar(pcm_1, ax=ax1, orientation='vertical', fraction=.1)
plt.show()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

... After correlation and cut on dt ...
Number of events acquired with PM2 after correlation and cut on dt: 0.141918 million


In [26]:
#After correlation, cut on dt and cut on electron/muon energy

mask = nrj2[1:][(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)] < nrj2[:-1][(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)]

#Muon as function of electron

fig, ax1 = plt.subplots(1, sharex=True)
arr_1,xe_1,ye_1 = np.histogram2d(nrj2[1:][(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)][mask],
                                 nrj2[:-1][(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)][mask],  
                                 bins =(500,100))
pcm_1 = ax1.pcolormesh(xe_1, ye_1,arr_1.T, cmap='rainbow', norm=LogNorm(vmin=arr_1.min()+1, vmax=arr_1.max()))
ax1.set_xlabel('Second Event Energy\n[Arbitrary Unit]')
ax1.set_ylabel('First Event Energy\n[Arbitrary Unit]')
ax1.set_title('2D Histogram\nFirst Event Energy as Function of Second Event Energy')
ax1.grid()
ax1.plot(np.linspace(1000, 250000, 10000),np.linspace(1000,250000, 10000),c='r')
ax1.set_xscale("log")
fig.colorbar(pcm_1, ax=ax1, orientation='vertical', fraction=.1)
plt.show()

#Muon and electron energy
plt.figure()
plt.hist(nrj2[1:][(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)][mask], bins=150, label="Second Event", alpha = 0.5)
#plt.hist(nrj2[:-1][(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)][mask], bins=150, label="First Event", alpha = 0.5)
#plt.hist(nrj2[(nrj2<nrj_max) & (np.append(dt2, np.nan)<dt_max)], bins=150, label="First Event", alpha = 0.5)
#plt.hist(nrj2[(nrj2<nrj_max) & (np.insert(dt2, 0, np.nan)<dt_max)], bins=150, label="Second Event", alpha = 0.5)
plt.xlabel('Energy [Arbitraty Unit]')
plt.ylabel('Occurrences')
plt.legend()
plt.grid()
plt.show()

#Electron as function of dt
fig, ax1 = plt.subplots(1, sharex=True)
arr_1,xe_1,ye_1 = np.histogram2d(dt2[(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)][mask], 
                                 nrj2[1:][(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)][mask],
                                 bins =(500,100))
pcm_1 = ax1.pcolormesh(xe_1, ye_1,arr_1.T, cmap='rainbow', norm=LogNorm(vmin=arr_1.min()+1, vmax=arr_1.max()))
ax1.set_xlabel('$\Delta t$ [µs]')
ax1.set_ylabel('Second Event Energy\n[Arbitrary Unit]')
ax1.set_title('2D Histogram\nSecond Event Energy as function of $\Delta t$')
ax1.grid()
fig.colorbar(pcm_1, ax=ax1, orientation='vertical', fraction=.1)
plt.show()

#Muon as function of dt
fig, ax1 = plt.subplots(1, sharex=True)
arr_1,xe_1,ye_1 = np.histogram2d(dt2[(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)][mask], 
                                 nrj2[:-1][(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)][mask],
                                 bins =(500,100))
pcm_1 = ax1.pcolormesh(xe_1, ye_1,arr_1.T, cmap='rainbow', norm=LogNorm(vmin=arr_1.min()+1, vmax=arr_1.max()))
ax1.set_xlabel('$\Delta t$ [µs]')
ax1.set_ylabel('First Event Energy\n[Arbitrary Unit]')
ax1.set_title('2D Histogram\nFirst Event Energy as function of $\Delta t$')
ax1.grid()
fig.colorbar(pcm_1, ax=ax1, orientation='vertical', fraction=.1)
plt.show()

In [13]:
#dt Analysis 

dt_max = 20
dt = dt2[(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)][mask]
print("Number of events acquired with PM2 after correlation cut on dt and energies:", len(dt)/10**6, "million")

N_bins = 100
hist = np.histogram(dt, bins = N_bins)
x = hist[1][1:]-(hist[1][1]-hist[1][0])/2
y = hist[0]
y_err = np.sqrt(y)

#Fit----------------------------------------------------

#With one exponential
popt, cov = curve_fit(BF.exp1, x, y, sigma=y_err, absolute_sigma = True, p0 = np.array([20000, 2, 400]))
N0 = popt[0]
lam = popt[1]
C = popt[2]
lam_err = cov[1,1]

chi2 = BF.chi2_norm(y, BF.exp1(x, N0, lam, C), y_err, 3)[0]
chi2_err = BF.chi2_norm(y, BF.exp1(x, N0, lam, C), y_err, 3)[1]
print("... Fit with one exponential ...")
print("chi2 = ", chi2, '+/-', chi2_err)
print('\n')

#With two exponentials
popt, cov = curve_fit(BF.exp2, x, y, sigma=y_err, absolute_sigma = True, p0 = np.array([20000, 2, 2, 400]))
N0 = popt[0]
lam1 = popt[1]
lam1_err = cov[1,1]
lam2 = popt[2]
lam2_err = cov[2,2]
CC = popt[3]

chi2 = BF.chi2_norm(y, BF.exp2(x, N0, lam1, lam2, CC), y_err, 4)[0]
chi2_err = BF.chi2_norm(y, BF.exp2(x, N0, lam1, lam2, CC), y_err, 4)[1]
print("... Fit with two exponentials ...")
print("chi2 = ", chi2, '+/-', chi2_err)

#-------------------------------------------------------

plt.figure()
plt.hist(dt2[(dt2<dt_max) & (nrj2[:-1]<0.25e6) & (nrj2[1:]<0.25e6)][mask], bins = N_bins, alpha = 0.5)
plt.errorbar(x, y, yerr = y_err, linestyle = 'None', fmt='o', capsize=4, c='k', label = 'Data')
plt.plot(np.linspace(min(dt), max(dt), 100), BF.exp1(np.linspace(min(dt), max(dt), 100), N0, lam, C), 
                     label = 'Fit with one exponential')
plt.plot(np.linspace(min(dt), max(dt), 100), BF.exp2(np.linspace(min(dt), max(dt), 100), N0, lam1, lam2, CC), 
                     label = 'Fit with two exponentials')
plt.xlabel("$\Delta t$ [µs]")
plt.ylabel("Occurrences")
plt.yscale('log')
plt.legend()
plt.show()

Number of events acquired with PM2 after correlation cut on dt and energies: 0.082345 million
... Fit with one exponential ...
chi2 =  0.9213510978212853 +/- 0.1435916317235476


... Fit with two exponentials ...
chi2 =  0.8395292924120423 +/- 0.14433756729740643


/home/hind/Documents/Cours/Cours M2/TL/BigMuoneXperiment/BigFunctions.py:5: RuntimeWarning: overflow encountered in exp
  def exp2(t, N0, lam1, lam2, C) : return N0*np.exp(-lam1*t) + 1.3475*N0*np.exp(-lam2*t) + C
/home/hind/Documents/Cours/Cours M2/TL/BigMuoneXperiment/BigFunctions.py:5: RuntimeWarning: overflow encountered in multiply
  def exp2(t, N0, lam1, lam2, C) : return N0*np.exp(-lam1*t) + 1.3475*N0*np.exp(-lam2*t) + C


In [14]:
tau = 1/lam
tau_err = lam_err/lam**2

tau1 = 1/lam1
tau1_err = lam1_err/lam1**2

tau2 = 1/lam2
tau2_err = lam2_err/lam2**2

print("... Fit with one exponential ...")
print('tau = ', tau, '+/-', tau_err, 'µs')
print('\n')
print('... Fit with two exponentials ...')
print('tau1 = ', tau1, '+/-', tau1_err, 'µs')
print('tau2 = ', tau2, '+/-', tau2_err, 'µs')

... Fit with one exponential ...
tau =  2.0228443522651443 +/- 2.95613008721786e-05 µs


... Fit with two exponentials ...
tau1 =  1.613252994185311 +/- 0.0020310659746548684 µs
tau2 =  2.2815507447087504 +/- 0.000299287269110054 µs


In [15]:
#dt Analysis 

dt_max = 20
dt = dt2[dt2 < dt_max]

N_bins = 45
hist = np.histogram(dt, bins = N_bins)
x = hist[1][1:]-(hist[1][1]-hist[1][0])/2
y = hist[0]
y_err = np.sqrt(y)

#Fit----------------------------------------------------

#With one exponential
popt, cov = curve_fit(BF.exp1, x, y, sigma=y_err, absolute_sigma = True, p0 = np.array([20000, 2, 400]))
N0 = popt[0]
lam = popt[1]
C = popt[2]
lam_err = cov[1,1]

chi2 = BF.chi2_norm(y, BF.exp1(x, N0, lam, C), y_err, 3)[0]
chi2_err = BF.chi2_norm(y, BF.exp1(x, N0, lam, C), y_err, 3)[1]
print("... Fit with one exponential ...")
print("chi2 = ", chi2, '+/-', chi2_err)
print('\n')

#With two exponentials
popt, cov = curve_fit(BF.exp2, x, y, sigma=y_err, absolute_sigma = True, p0 = np.array([20000, 2, 2, 400]))
N0 = popt[0]
lam1 = popt[1]
lam1_err = cov[1,1]
lam2 = popt[2]
lam2_err = cov[2,2]
CC = popt[3]

chi2 = BF.chi2_norm(y, BF.exp2(x, N0, lam1, lam2, CC), y_err, 4)[0]
chi2_err = BF.chi2_norm(y, BF.exp2(x, N0, lam1, lam2, CC), y_err, 4)[1]
print("... Fit with two exponentials ...")
print("chi2 = ", chi2, '+/-', chi2_err)

#-------------------------------------------------------

plt.figure()
plt.hist(dt2[dt2 < dt_max], bins = N_bins, alpha = 0.5)
plt.errorbar(x, y, yerr = y_err, linestyle = 'None', fmt='o', capsize=4, c='k', label = 'Data')
plt.plot(np.linspace(min(dt), max(dt), 100), BF.exp1(np.linspace(min(dt), max(dt), 100), N0, lam, C), 
                     label = 'Fit with one exponential')
plt.plot(np.linspace(min(dt), max(dt), 100), BF.exp2(np.linspace(min(dt), max(dt), 100), N0, lam1, lam2, CC), 
                     label = 'Fit with two exponentials')
plt.xlabel("$\Delta t$ [µs]")
plt.ylabel("Occurrences")
plt.yscale('log')
plt.legend()
plt.show()

... Fit with one exponential ...
chi2 =  1.1402428512628409 +/- 0.21821789023599236


... Fit with two exponentials ...
chi2 =  1.0648136814293978 +/- 0.2208630521496931


In [16]:
tau = 1/lam
tau_err = lam_err/lam**2

tau1 = 1/lam1
tau1_err = lam1_err/lam1**2

tau2 = 1/lam2
tau2_err = lam2_err/lam2**2

print("... Fit with one exponential ...")
print('tau = ', tau, '+/-', tau_err, 'µs')
print('\n')
print('... Fit with two exponentials ...')
print('tau1 = ', tau1, '+/-', tau1_err, 'µs')
print('tau2 = ', tau2, '+/-', tau2_err, 'µs')

... Fit with one exponential ...
tau =  2.032800984498693 +/- 1.882290381447346e-05 µs


... Fit with two exponentials ...
tau1 =  2.3120490744600186 +/- 0.0007970524952133977 µs
tau2 =  1.804325308957612 +/- 0.0010553535888393655 µs


In [17]:
#chi2

plt.figure()
N_bins = np.array(range(10,301))
chi2_lst = []

for n in N_bins :
    hist = np.histogram(dt, bins = n)
    x = hist[1][1:]-(hist[1][1]-hist[1][0])/2
    y = hist[0]
    y_err = np.sqrt(y)

    popt, cov = curve_fit(BF.exp2, x, y, sigma=y_err, absolute_sigma = True, p0 = np.array([20000, 2, 2, 400]))
    N0 = popt[0]
    lam1 = popt[1]
    lam2 = popt[2]
    CC = popt[3]

    chi2 = BF.chi2_norm(y, BF.exp2(x, N0, lam1, lam2, CC), y_err, 4)[0]
    chi2_err = BF.chi2_norm(y, BF.exp2(x, N0, lam1, lam2, CC), y_err, 4)[1]

    chi2_lst.append(chi2)

    plt.errorbar(n, chi2, yerr = chi2_err, linestyle = 'None', fmt='o', capsize=4, c='k')
    plt.scatter(n, chi2, c='k')

plt.xlabel("Number of bins")
plt.ylabel("Reduced $\chi^2$")
plt.show()
    

In [18]:
chi2_lst = np.array(chi2_lst)
print(min(np.abs(1-chi2_lst))+1)

print(np.argwhere(chi2_lst == min(np.abs(1-chi2_lst))+1))

print(chi2_lst[35])
print(N_bins[35])

1.004758666693851
[[43]]
1.0648136814293978
45
